# Advanced Econometrics: Homework 1

5.11.2020

Barbora Jakubová  
Natálie Kolářová  
Barbara Livorová

---

# Problem 1

Weibull distribution has the following density:

$$f(x) = \alpha \beta x^{\beta-1} e^{-\alpha x^\beta}, x \geq 0, \alpha, \beta > 0.$$

(a) Obtain log-likelihood function for a random sample of $n$ observations.

* From the likelihood function $$L(\alpha, \beta) = \prod_{i=1}^n \alpha \beta x_i^{\beta - 1} e^{- \alpha x_i^{\beta}}$$ we can obtain a log-likelihood function
* $$lnL(\alpha,\beta)= \sum_{i=1}^n ln[\alpha \beta x_i^{\beta - 1} e^{- \alpha x_i^{\beta}}] = \sum_{i=1}^n [ln\alpha + ln\beta + (\beta - 1) lnx_i - \alpha x_i^{\beta} ln(e)] = n * ln\alpha + n * ln\beta + (\beta - 1) \sum_{i=1}^n lnx_i - \alpha \sum_{i=1}^n x_i^{\beta}$$

(b) Obtain the necessary condition equations for $\alpha$ and $\beta$. Note that we get the solution for $\alpha$ in
terms of data $x$ and $\beta$. While for the second equation we get only an implicit solution for $\beta$.
How would you continue to obtain the maximum likelihood estimators?

* The necessary conditions for maximizing the log-likelihood function (called FOC as well) are that the partial derivatives are equal to zero $$\frac{\delta L(\alpha, \beta)}{\delta\alpha} = \frac{n}{\alpha} - \sum_{i=1}^{n} x_i^{\beta} = 0$$
$$\frac{\delta L(\alpha, \beta)}{\delta\beta} = \frac{n}{\beta} + \sum_{i=1}^{n} lnx_i - \alpha \sum_{i=1}^{n} x_i^{\beta} lnx_i = 0$$
* From the necessary conditions we can easily obtain the value of $\alpha$
$$\frac{n}{\alpha} = \sum_{i=1}^{n} x_i^{\beta}$$
$$\alpha = \frac{n}{\sum_{i=1}^{n} x_i^{\beta}}$$

* In oder to calculate $\beta$ we substitute for $\alpha$
$$\frac{n}{\beta} - \alpha \sum_{i=1}^{n} x_i^{\beta} lnx_i = \sum_{i=1}^{n} lnx_i$$
$$\frac{n}{\beta} - \frac{n}{\sum_{i=1}^{n} x_i^{\beta}} \sum_{i=1}^{n} x_i^{\beta} lnx_i = \sum_{i=1}^{n} lnx_i$$
$$\frac{n}{\beta} = n \sum_{i=1}^{n} lnx_i - \sum_{i=1}^{n} lnx_i$$
$$\frac{n}{\beta} = (n-1) \sum_{i=1}^{n} lnx_i$$
$$\beta = \frac{n}{(n-1)\sum_{i=1}^{n} lnx_i}$$ 

(c) Obtain a Hessian matrix of the log-likelihood with respect to $\alpha$ and $\beta$.

$$H = \begin{pmatrix}
\frac{\delta lnL(\alpha, \beta)}{\delta \alpha^2} & \frac{\delta lnL(\alpha, \beta)}{\delta\alpha \delta\beta}\\
\frac{\delta lnL(\beta, \alpha)}{\delta\alpha \delta\beta} & \frac{\delta lnL(\alpha, \beta)}{\delta \beta^2}
\end{pmatrix} = \begin{pmatrix} -\frac{n}{a^2} & - \sum_{i = 1}^{n} lnx_i * x_i^{\beta} \\ - \sum_{i = 1}^{n} lnx_i * x_i^{\beta} & -\frac{n}{\beta^2} - \alpha \sum_{i=1}^{n} ln^2x_i * x_i^{\beta}\end{pmatrix}$$

You are provided with sample data generated by the Weibull distribution in 'problem_1_dataset.Rdata'.  

(d) Obtain maximum likelihood estimates of $\alpha$ and $\beta$, and estimate the asymptotic covariance matrix for the estimates.

In [2]:
library('maxLik')
library('lmtest')
load("problem_1_dataset.Rdata")

In [5]:
ls()

[1] "w_data"

In [6]:
str(w_data)

 num [1:5000] 7.71 4.07 9.55 9.2 4.28 ...


In [14]:
n <- length(w_data)
x <- w_data
logLikFun <- function(param) {
  if (param[1] <= 0 || param[2] <= 0) {
    return(NA)
  }else{
  return(n*log(param[1]) + n*log(param[2]) + (param[2]-1)*sum(log(x)) - param[1]*sum(x^param[2]))
}}
Hessian <- function(param){
    if(param[1] <= 0 | param[2] <= 0) return(NA)
    alpha <- param[1]
    beta <- param[2]
    h11 <- (-n)/alpha^2
    h12 <- -sum(log(x)*x^beta)
    h21 <- -sum(log(x)*x^beta)
    h22 <- ((-n)/(beta^2)) - alpha * sum((log(x)^2)*(x^beta))
    h_matrix <- matrix(c(h11, h12, h21, h22), nrow = 2, ncol = 2)
    return(h_matrix)
    }
mle <- maxLik(logLikFun, start = c(alpha = 0.01, beta = 0.01))
summary(mle)
vcov(mle)

--------------------------------------------
Maximum Likelihood estimation
Newton-Raphson maximisation, 12 iterations
Return code 2: successive function values within tolerance limit
Log-Likelihood: -18605.46 
2  free parameters
Estimates:
      Estimate Std. error t value Pr(> t)    
alpha 0.062975   0.002381   26.45  <2e-16 ***
beta  1.013935   0.011145   90.98  <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
--------------------------------------------

,alpha,beta
alpha,5.668377e-06,-2.460789e-05
beta,-2.460789e-05,1.242100e-04


In [15]:
mle2 <- maxLik(logLikFun, hess = Hessian, start = c(alpha = 0.01, beta = 0.01))
summary(mle2)
vcov(mle2)

--------------------------------------------
Maximum Likelihood estimation
Newton-Raphson maximisation, 12 iterations
Return code 2: successive function values within tolerance limit
Log-Likelihood: -18605.46 
2  free parameters
Estimates:
      Estimate Std. error t value Pr(> t)    
alpha 0.062974   0.002381   26.45  <2e-16 ***
beta  1.013935   0.011147   90.96  <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
--------------------------------------------

,alpha,beta
alpha,5.669403e-06,-2.461422e-05
beta,-2.461422e-05,1.242472e-04


In [23]:
vcov(mle2)

,alpha,beta
alpha,5.668377e-06,-2.460789e-05
beta,-2.460789e-05,1.242100e-04


(e) Obtain maximum likelihood estimate of $\alpha$ under the hypothesis that $\beta$ = 1.

In [26]:
logLikFun <- function(param) {
  if (param[1] <= 0) {
    return(NA)
  }else{
  return(n*log(param) + n*log(1) + (1-1)*sum(log(x)) - param[1]*sum(x^1))
}}
mle3 <- maxLik(logLikFun, start = c(alpha = 0.01))
summary(mle3)

--------------------------------------------
Maximum Likelihood estimation
Newton-Raphson maximisation, 7 iterations
Return code 1: gradient close to zero
Log-Likelihood: -18606.25 
1  free parameters
Estimates:
       Estimate Std. error t value Pr(> t)    
alpha 0.0657925  0.0009305   70.71  <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
--------------------------------------------

(f) Carry out Wald test with $H_0$ : $\beta$ = 1.

In [27]:
waldtest(mle3, mle2)

Res.Df,Df,Chisq,Pr(>Chisq)
NA,NA,NA,NA
NA,1,8276.822,0


* Tedy by měl být lepší ten původní model

(g) Carry out likelihood ratio test with $H_0$ : $\beta$ = 1.

In [29]:
lrstat <- 2*(logLik(mle3) - logLik(mle2))
pchisq(lrstat, df = 1, lower.tail = F)

[1] 1

(h) Carry out Lagrange multiplier test with $H_0$ : $\beta$ = 1.

---

# Problem 2